In [1]:
import numpy
import random

In [2]:
def copying_data(T=30, n_data=300*64, n_sequence=10, batch_size=64, make_rand=False, shuffle=False, drop_last=False):
    seq = numpy.random.randint(1, high=9, size=(n_data, n_sequence))
    zeros1 = numpy.zeros((n_data, T-1))
    zeros2 = numpy.zeros((n_data, T))
    marker = 9 * numpy.ones((n_data, 1))
    zeros3 = numpy.zeros((n_data, n_sequence))

    x = numpy.concatenate((seq, zeros1, marker, zeros3), axis=1).astype('int32')
    y = numpy.concatenate((zeros3, zeros2, seq), axis=1).astype('int64')

    x = numpy.expand_dims(x, axis=-1)
    y = numpy.expand_dims(y, axis=-1)
    
    return x, y

In [3]:
random.seed(0)

train_x, train_y = copying_data(T=50)
test_x, test_y   = copying_data(T=200)

# Model comparison

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import LSTMCell
from tfRIM import RIMCell

## LSTM model

In [5]:
lstmcell = LSTMCell(600)

In [14]:
inputs = tf.keras.Input([None, 1])
rnnout = tf.keras.layers.RNN(lstmcell, return_sequences=True)(inputs)
output = tf.keras.layers.Dense(20)(rnnout)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
cost = tf.keras.losses.SparseCategoricalCrossentropy()

model_lstm = tf.keras.Model(inputs, output)
model_lstm.compile(optimizer=opt, loss=cost)

In [ ]:
model_lstm.fit(train_x, train_y, batch_size=64, epochs=5)

## RIMs model

In [15]:
rimcell = RIMCell(units=500, nRIM=5, k=2,
                num_input_heads=4, input_key_size=32, input_value_size=32, input_query_size=32, input_keep_prob=0.5,
                num_comm_heads=4, comm_key_size=32, comm_value_size=32, comm_query_size=32, comm_keep_prob=0.5)

In [16]:
inputs = tf.keras.Input([None, 1])
rnnout = tf.keras.layers.RNN(rimcell, return_sequences=True)(inputs)
output = tf.keras.layers.Dense(20)(rnnout)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
cost = tf.keras.losses.SparseCategoricalCrossentropy()

model_rims = tf.keras.Model(inputs, output)
model_rims.compile(optimizer=opt, loss=cost)

In [8]:
# model_rims = tf.keras.Sequential()
# model_rims.add(tf.keras.layers.Input([None, 1]))
# model_rims.add(tf.keras.layers.RNN(rimcell, return_sequences=True))
# model_rims.add(tf.keras.layers.Dense(20))
# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# cost = tf.keras.losses.SparseCategoricalCrossentropy()
# model_rims.compile(optimizer=opt, loss=cost)

In [ ]:
model_rims.fit(train_x, train_y, batch_size=64, epochs=5)